In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon
import scipy.stats as st
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.metrics import roc_curve

In [ ]:
# Question 1 #
df = pd.read_csv('all_weeks.csv')
df.columns=['week','cases']
print(df.head())

In [ ]:
# 1A # 
mu = 1/100
# draw from expoential for gamma
lower, upper = 1, 2
mean, sigma = 1.9, 2.0

#gamma = 1/expon.rvs(scale=2, size = 1)[0] # could not get this to work well 
gamma = 1/2
#isolate exponential period 
I = df['cases'].values[2:17]
N_t = df['week'].values[2:17]
# draw from truncated normal distribution for underestimated cases
under_est = stats.truncnorm.rvs(
    (lower - mean) / sigma, (upper - mean) / sigma, loc=mean, scale=sigma,size=len(I)) #% https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.truncnorm.html

I = I * under_est

I = np.log(I)
mod = sm.OLS(I, N_t)
res = mod.fit()
        
# confidence interval for slope distribution        
#ci = st.norm.interval(confidence=0.95, loc=np.mean(slopes), scale=st.sem(slopes))
ci =  res.conf_int(0.05)[0]
print("CI for slope: ", ci)
# analytical calculation for slope
#slope, intercept = np.polyfit(I, N_t, 1)
slope = res.params
print("Calculated slope: ", slope)



In [ ]:

r_naughts = []
r_naughts.append(1+(ci[0]/(gamma + mu)))
r_naughts.append(1+(ci[1]/(gamma + mu)))
ci = st.norm.interval(confidence=0.95, loc=np.mean(r_naughts), scale=st.sem(r_naughts))
#ci = st.norm.interval(confidence=0.95, loc=np.mean(r_naughts), scale=st.sem(r_naughts))
print("CI for r_o: ", ci)

r_n = 1 + (slope/(gamma + mu))
print("Calculated r_o: ", r_n)

# R0 = 1.86

fig, ax = plt.subplots()
ax.plot(N_t, I, label = 'I', color = 'red', linestyle="",marker="o")
#l1 = plt.plot()
#plt.ylim(0,1)
#plt.xlim(0,100)
plt.xlabel('Time')
plt.ylabel('log[I]')
plt.title('Exponential Phase of Infection')
#plt.yscale('log')
plt.legend()
plt.show()


In [ ]:
# 1B # 
mu = 1/100
gamma = 1/2

print("Calculated seroprevalance: ", (517/1000))

ci = [0.517 - (1.96 * np.sqrt((0.517 * (1-0.517))/1000)), 0.517 + (1.96 * np.sqrt((0.517 * (1-0.517))/1000))]
print("CI for seroprevalance: ", ci) 

print("Calculated R_0: ", 1/(1-(517/1000)))

r_naughts = [1/(1-ci[0]), 1/(1-ci[1])] 
ci = st.norm.interval(confidence=0.95, loc=np.mean(r_naughts), scale=st.sem(r_naughts))
print("CI for R_0: ", ci) 

In [ ]:
# 1C # 
mu = 1/100
gamma = 1/2
#how to calculate i_eq, where to use prevalance
inf_equil = 7/1000
print("Calculated I_eq: ", inf_equil)

ci = [inf_equil - (1.96 * np.sqrt((inf_equil * (1-inf_equil))/1000)), inf_equil + \
      (1.96 * np.sqrt((inf_equil * (1-inf_equil))/1000))]
print("CI for seroprevalance: ", ci) 


print("Calculated R_0: ", (1/(1 - inf_equil * ((gamma/mu) + 1))))
ci = [(1/(1 - ci[0] * ((gamma/mu) + 1))), (1/(1 - ci[1] * ((gamma/mu) + 1)))] 
print("CI for R_0: ", ci)                          
                        
    

In [ ]:
# Question 2 #
se = 0.90
sp = 0.98

In [ ]:
# 2A # 
npos = 39
N = 100
num = (npos/N) - (1 - sp) 
den = (se + sp -1 )
prev = num/den
print("Calculated prevalance: ", prev)

In [ ]:
# 2B # 
ci = [0.420 - (1.96 * np.sqrt((0.420 * (1-0.420))/100)), 0.420+ (1.96 * np.sqrt((0.420 * (1-0.420))/100))]
print("CI for prevalance: ", ci) 
#0.324, .517 
#Z = 1.96

In [ ]:
# 2C # 
npos = 18
N = 50
num = (npos/N) - (1 - sp) 
den = (se + sp -1 )
prev = num/den
print("Calculated prevalance: ", prev)

ci = [0.386- (1.96 * np.sqrt((0.386 * (1-0.386))/50)), 0.386+ (1.96 * np.sqrt((0.386 * (1-0.386))/50))]
print("CI for prevalance: ", ci) 

In [ ]:
# 2D # 
npos = 39 + 18
N = 100 + 50
num = (npos/N) - (1 - sp) 
den = (se + sp -1 )
prev = num/den
print("Calculated prevalance: ", prev)

ci = [0.409 - (1.96 * np.sqrt((0.409 * (1-0.409))/150)), 0.409+ (1.96 * np.sqrt((0.409 * (1-0.409))/1001500))]
print("CI for prevalance: ", ci) 

In [ ]:
# 2E # 
phi = prev*se + (1-prev)*(1-sp)
print("Probability test comes back positive: ", phi)
p_positive = (se * prev)/(phi)
print("Probability truly positive given test positive: ", p_positive)

In [ ]:
# Question 3 #
#HW4 Q3 neg.csv: The assay values associated with a set of negative controls.
#• HW4 Q3 pos.csv: The assay values associated with a set of positive controls.
#• HW4 Q3 data.csv: The assay values associated with your prevalence study in the population.

df_neg = pd.read_csv('HW4_Q3_neg.csv')
print(df_neg.head())

df_pos = pd.read_csv('HW4_Q3_pos.csv')
print(df_pos.head())

df_data = pd.read_csv('HW4_Q3_data.csv')
print(df_data.head())

In [ ]:
# 3A # 

fig,ax = plt.subplots(figsize = (2,4))

jitter_pos = np.random.rand(len(df_pos))
ax.scatter(jitter_pos,df_pos,alpha=0.25,color='black', label = 'Pos')

jitter_neg = np.random.rand(len(df_neg))
ax.scatter(-1.5+jitter_neg,df_neg,alpha=0.25,color='red',label = 'Neg')

jitter_data = np.random.rand(len(df_data))
ax.scatter(1.5+jitter_data,df_data,alpha=0.25,color='b', label = 'Data')

#plt.legend(loc = 'upper left')
plt.show()

In [ ]:
def J(c):
    TP = df_pos[df_pos > c].dropna().values.flatten()
    FN = df_pos[df_pos < c].dropna().values.flatten()
    FP = df_neg[df_neg > c].dropna().values.flatten()
    TN = df_neg[df_neg < c].dropna().values.flatten()
    se = len(TP)/(len(TP)+len(FN))
    sp = len(TN)/(len(TN)+len(FP))
    return se + sp - 1 
def se(c):
    TP = df_pos[df_pos > c].dropna().values.flatten()
    FN = df_pos[df_pos < c].dropna().values.flatten()
    se = len(TP)/(len(TP)+len(FN))
    return se
def sp(c):
    FP = df_neg[df_neg > c].dropna().values.flatten()
    TN = df_neg[df_neg < c].dropna().values.flatten()
    sp = len(TN)/(len(TN)+len(FP))
    return sp
def raw_prev(c):
    pos = df_data[df_data > c].dropna().values.flatten()
    neg = df_data[df_data < c].dropna().values.flatten()
    return pos/neg
def adj_prev(c):
    spec = sp(c)
    sens = se(c)
    npos = len(df_data[df_data > c].dropna().values.flatten())
    N = len(df_data.dropna().values.flatten())
    return (npos/N - (1 - spec))/(sens + spec - 1)

In [ ]:
from operator import itemgetter
results = []
for i in range(25):
    results.append(J(i))
max_value = max(results)

You_index = results.index(max_value)
print("Youden Index: ", You_index)

In [ ]:
#3c#
tpr = []
fpr = []
for i in range(25):
    tpr.append(se(i))
    fpr.append(1-sp(i))
    plt.plot(fpr, tpr, 'b')
plt.plot(fpr[15],tpr[15], marker='o', c='r', label = 'Youden')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('ROC')

In [ ]:
#3C# 
theta = []
fpr = []
for i in range(3,30):
    theta.append(adj_prev(i))
x = range(len(theta))
plt.plot(x, theta, 'b')
plt.plot(x[12], theta[12],c='r', marker='o', label = 'Youden')
plt.xlabel('Cutoff')
plt.ylabel('Corrected Prevalance')
plt.legend()